In [7]:
# need this import so Term and Sentence
# can talk about each other
from __future__ import annotations

import csv

In [56]:
class Term:
    """This basically represents all the data needed
    to generate an Anki flaschard for one word / phrase
    in the target language
    """
    text: str  # the word / phrase in the target language.
    translation: str  # the English translation of the word / phrase.
    sentence_ids: list[str]  # list of Tatoeba sentence ids for
                             # the example sentences that go with this term.
                             # This should maybe be a list of ints instead of strs.
    sentences: list[Sentence]  # Objects representing the example sentences.

    def __init__(self, text, translation, sentence_ids):
        self.text = text
        self.translation = translation
        self.sentence_ids = sentence_ids
        self.sentences = []  # we will fill this later.

    def print(self):
        print("Term(\n    text: {0}\n    translation: {1}\n    sentence_ids: {2}\n    sentences: {3})".format(
            self.text, self.translation, self.sentence_ids, self.sentences))

class Sentence:
    tatoeba_id: str  # Tatoeba sentence id for this sentence.
                     # Should maybe be a str.
    text: str  # The text of the sentence.
    translations: list[str] | None  # English translations of this sentence.
                                    # Not all sentences will have translations,
                                    # so this value could be None.
    audio_id: str | None  # Tatoeba audio id for this sentence. Not all
                   # sentences will have audios. If there is no
                   # audio, this will be None. This should maybe be an int.
    parent_term: Term  # A reference to the vocab term that this sentence
                       # serves as an example for. This is useful in the
                       # data generation part.

    def __init__(self, parent_term, tatoeba_id, text, translations=None, audio_id=None):
        self.parent_term = parent_term
        self.tatoeba_id = tatoeba_id
        self.text = text
        self.translations = translations
        self.audio_id = audio_id

In [57]:
def process_tsv(process_fn, filepath_str, encoding='utf-8-sig'):
    """Call ROW_FN on each row of a TSV file"""
    with open(filepath_str, newline='', encoding=encoding) as f:
        reader = csv.reader(f, delimiter='\t')
        return process_fn(reader)

In [58]:
def process_vocab_list(tsv_reader, skip_header=True):
    terms = []
    header = next(tsv_reader)
    for row in tsv_reader:
        word_de, word_en = row[:2]
        sentence_ids = [s for s in row[2:] if s]
        one_term = Term(word_de, word_en, sentence_ids)
        terms.append(one_term)
    return terms

In [59]:
vocab_list_file = './NW_7_Days-and-times - Tageszeiten.tsv'
terms = process_tsv(process_vocab_list, vocab_list_file)

In [60]:
for t in terms:
    t.print()

Term(
    text: der Abend, die Abende
    translation: evening
    sentence_ids: ['998']
    sentences: [])
Term(
    text: das Abendessen, die Abendessen
    translation: dinner
    sentence_ids: ['341418']
    sentences: [])
Term(
    text: am Morgen
    translation: in the morning
    sentence_ids: ['347433']
    sentences: [])
Term(
    text: jemanden an|rufen
    translation: to call someone
    sentence_ids: ['873078']
    sentences: [])
Term(
    text: def Apfel, die Äpfel
    translation: apple
    sentence_ids: ['367095']
    sentences: [])
Term(
    text: etwas auf|räumen
    translation: to clean up something; to tidy up something
    sentence_ids: ['618818']
    sentences: [])
Term(
    text: aus|gehen
    translation: to go out
    sentence_ids: ['882822']
    sentences: [])
Term(
    text: auf|stehen
    translation: to get up
    sentence_ids: ['365619']
    sentences: [])
Term(
    text: etwas ein|kaufen
    translation: to buy/shop for something
    sentence_ids: ['356